In [1]:
import yaml  # Import yaml library to read yaml files

# Read the data.yaml file from Dataset folder
with open('../Dataset/data.yaml', 'r') as file:  # Open file in read mode
    data = yaml.safe_load(file)  # Load yaml content into data variable
data

{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 5,
 'names': ['Looking_Forward',
  'Raising_Hand',
  'Reading',
  'Sleeping',
  'Turning_Around'],
 'roboflow': {'workspace': 'mohamed-hazem-abdelhady-ahmed',
  'project': 'student-action-recognition-9cibq',
  'version': 3,
  'license': 'CC BY 4.0',
  'url': 'https://universe.roboflow.com/mohamed-hazem-abdelhady-ahmed/student-action-recognition-9cibq/dataset/3'}}

In [2]:
import os
import cv2

# Get list of images in train directory
train_dir = '../Dataset/train/images'
train_images = sorted(os.listdir(train_dir))[:2]  # Get first 2 images
images = []
# Read and display first two images
for img_name in train_images:
    img_path = os.path.join(train_dir, img_name)
    images.append(img_path)


In [4]:
import numpy as np
def interpolate_frames_optical_flow(frame1_path: str, frame2_path: str, output_path: str, timestep: float = 0.5, show_results: bool = False):
    """
    Interpolates two frames using OpenCV's Farneback optical flow.

    Args:
        frame1_path (str): Path to the first input image frame.
        frame2_path (str): Path to the second input image frame.
        output_path (str): Path to save the interpolated output image.
        timestep (float): The position of the interpolated frame between frame1 and frame2.
                          0.5 means exactly in the middle. (Value between 0 and 1)
    """

    # 1. Load Images
    img1 = cv2.imread(frame1_path)
    img2 = cv2.imread(frame2_path)

    if img1 is None or img2 is None:
        print("Error: Could not load one or both images. Check paths.")
        return

    # Ensure images have the same dimensions
    if img1.shape != img2.shape:
        print("Error: Input images must have the same dimensions for optical flow.")
        # Resize img2 to match img1 if necessary, or vice-versa
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))
        print("Resized img2 to match img1 dimensions.")

    height, width, _ = img1.shape

    # 2. Convert to Grayscale
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 3. Calculate Optical Flow (Farneback method)
    # The flow 'flow_fwd' represents motion from gray1 to gray2
    flow_fwd = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # 4. Generate Coordinate Grids
    # Create a grid of x and y coordinates
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Convert to float32 for cv2.remap
    x_float = x.astype(np.float32)
    y_float = y.astype(np.float32)

    # 5. Warp Frames
    # Map for img1: (x,y) in result should come from (x - timestep * flow_x, y - timestep * flow_y) in img1
    map_x_fwd = x_float - timestep * flow_fwd[..., 0]
    map_y_fwd = y_float - timestep * flow_fwd[..., 1]
    warped_img1 = cv2.remap(img1, map_x_fwd, map_y_fwd, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    # Map for img2: (x,y) in result should come from (x + (1-timestep) * flow_x, y + (1-timestep) * flow_y) in img2
    # The flow_fwd is from img1 to img2. So to go from img2 *backwards* to img1's position, we need to add the flow.
    map_x_bwd = x_float + (1 - timestep) * flow_fwd[..., 0]
    map_y_bwd = y_float + (1 - timestep) * flow_fwd[..., 1]
    warped_img2 = cv2.remap(img2, map_x_bwd, map_y_bwd, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    # 6. Blend Warped Frames
    # Simple averaging
    interpolated_frame = cv2.addWeighted(warped_img1, 1 - timestep, warped_img2, timestep, 0)

    # 7. Save/Display Result
    cv2.imwrite(output_path, interpolated_frame)

    # Optional: Display images (requires GUI environment)
    if show_results:
        cv2.imshow("Frame 1", img1)
        cv2.imshow("Frame 2", img2)
        cv2.imshow("Interpolated Frame", interpolated_frame)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


In [5]:
# interpolate_frames_optical_flow(images[0],images[1],'imgnew.jpg')

In [3]:
image_path = '../Dataset/train/images'
data_list = os.listdir(image_path)
data_list = [i for i in data_list if i.endswith('.jpg')]

In [4]:
import pandas as pd
data_list_labels = [i.split('_')[0] for i in data_list]
df = pd.DataFrame(data_list_labels)
df = df.value_counts()
#df.to_csv('../Dataset/class_count.csv')

In [5]:
import shutil
from IPython.display import clear_output
class_name = ""
count_frames = 1
interpolated_frames = 0
class_labels = {}
for i in range(len(data_list)):
    if class_name == data_list[i].split('_')[0]:
        count_frames+=1
    else:
        class_name = data_list[i].split('_')[0]
        class_labels[class_name] = []
        count_frames = 1
        interpolated_frames = 0
    
    try:
        frame_number = data_list[i].split('_')[3].split('frame')[1]
    except IndexError:
        frame_number = data_list[i].split('_')[3]
    total_frames = interpolated_frames+count_frames
    new_image_name = class_name+'_'+str(int(total_frames))+'.jpg'
    if df[class_name] >= 5:
        # shutil.copy(f'../Dataset/train/images/{data_list[i]}', f'../Dataset/PPDataset/train/images/{new_image_name}')
        # shutil.copy(f'../Dataset/train/labels/{data_list[i][:-4]}.txt', f'../Dataset/PPDataset/train/labels/{new_image_name.split(".")[0]}.txt')
        class_labels[class_name].append(new_image_name)
    if (5<=df[class_name] < 8) and (count_frames < df[class_name]) and ((total_frames)<4):
        interpolated_frames += 1
        # interpolate_frames_optical_flow(f'../Dataset/train/images/{data_list[i]}',
        #                                 f'../Dataset/train/images/{data_list[i+1]}',
        #                                 f'../Dataset/PPDataset/train/images/{new_image_name}')
        # shutil.copy(f'../Dataset/train/labels/{data_list[i][:-4]}.txt', f'../Dataset/PPDataset/train/labels/{new_image_name.split(".")[0]}.txt')
        class_labels[class_name].append(new_image_name)
    clear_output(wait = True)
    print(f'{i}/{len(data_list)}')
    

2261/2262


In [69]:
data_list

['around01_rgb_1_frame10_png_jpg.rf.4f039825b5c99ef547d56e2e39913e22.jpg',
 'around01_rgb_1_frame11_png_jpg.rf.cb75464e14591488780740e9af3da655.jpg',
 'around01_rgb_1_frame12_png_jpg.rf.4a1660715791730f7de0b3e7f8aefce3.jpg',
 'around01_rgb_1_frame13_png_jpg.rf.1e5d8729ed0a08b9bb1724c25b3335e8.jpg',
 'around01_rgb_1_frame1_png_jpg.rf.69a1667a34fbd878d4b334c6c14aeb08.jpg',
 'around01_rgb_1_frame2_png_jpg.rf.e5e815146a69b174b5956a6d33b44e09.jpg',
 'around01_rgb_1_frame3_png_jpg.rf.c26f34f0152d0f8d9038ae3b1f5c1055.jpg',
 'around01_rgb_1_frame4_png_jpg.rf.3e1b26d6a6f5639d586e54bbc7c631f7.jpg',
 'around01_rgb_1_frame5_png_jpg.rf.c759d711cba6a3dc5cb247e8cdd7e53a.jpg',
 'around01_rgb_1_frame9_png_jpg.rf.b28368e80aa3ebc06a981379cd573c56.jpg',
 'around02_rgb_1_frame103_png_jpg.rf.7aacd75118929ec2fcde786d7c39b609.jpg',
 'around02_rgb_1_frame104_png_jpg.rf.745fa139cc48a807f330741c78f07258.jpg',
 'around02_rgb_1_frame105_png_jpg.rf.00f457d31ee1559639d22fa74da2a571.jpg',
 'around02_rgb_1_frame106_pn

In [94]:
# import pickle
# # Dumping class_labels
# with open('../Dataset/class_labels.pickle', 'wb') as file:
#     pickle.dump(class_labels, file)
#     print(f"Object successfully pickled and saved to '../Dataset/class_labels.pickle'! ✨")

Object successfully pickled and saved to '../Dataset/class_labels.pickle'! ✨


In [97]:
samples = []
for class_label in class_labels.keys():
    class_type = ''.join(filter(str.isalpha, class_label))
    for i in range(len(class_labels[class_label])-8):
        sample_img_pahts = []
        for j in range(i,i+8):
            sample_img_pahts.append(class_labels[class_label][j])
        samples.append([class_type, class_label, sample_img_pahts])
samples

[['around',
  'around01',
  ['around01_1.jpg',
   'around01_2.jpg',
   'around01_3.jpg',
   'around01_4.jpg',
   'around01_5.jpg',
   'around01_6.jpg',
   'around01_7.jpg',
   'around01_8.jpg']],
 ['around',
  'around01',
  ['around01_2.jpg',
   'around01_3.jpg',
   'around01_4.jpg',
   'around01_5.jpg',
   'around01_6.jpg',
   'around01_7.jpg',
   'around01_8.jpg',
   'around01_9.jpg']],
 ['around',
  'around02',
  ['around02_1.jpg',
   'around02_2.jpg',
   'around02_3.jpg',
   'around02_4.jpg',
   'around02_5.jpg',
   'around02_6.jpg',
   'around02_7.jpg',
   'around02_8.jpg']],
 ['around',
  'around02',
  ['around02_2.jpg',
   'around02_3.jpg',
   'around02_4.jpg',
   'around02_5.jpg',
   'around02_6.jpg',
   'around02_7.jpg',
   'around02_8.jpg',
   'around02_9.jpg']],
 ['around',
  'around02',
  ['around02_3.jpg',
   'around02_4.jpg',
   'around02_5.jpg',
   'around02_6.jpg',
   'around02_7.jpg',
   'around02_8.jpg',
   'around02_9.jpg',
   'around02_10.jpg']],
 ['around',
  'arou

In [90]:
# samples = []
# print("Data either with small frame number or doesn't exist:")
# for class_label in class_labels:
#     if len(class_labels[class_label]) >= 8:
#         class_type = ''.join(filter(str.isalpha, class_label))
#         sample_data = []
#         for i in range(len(class_labels[class_label]) - 8):
#             for j in range(i, i + 8):
#                 sample_data.append(class_labels[class_label][j])
        
#         if sample_data:  # Only append if something was collected
#             samples.append([class_type, class_label, sample_data])
#         else:
#             print(class_label, "small frame number")
#     else:
#         print(class_label, "too short")

Data either with small frame number or doesn't exist:
around10 small frame number
around14 small frame number
around15 small frame number
around17 small frame number
forward05 too short
forward08 too short
forward11 too short
forward14 too short
forward15 too short
forward19 too short
forward21 too short
forward24 small frame number
hand05 small frame number
hand40 small frame number
read13 small frame number


In [98]:
samples_df = pd.DataFrame(samples, columns=['class_type', 'class_label', 'img_names']) #samples

In [99]:
sum(samples_df['img_names'].apply(lambda x: len(x))==0)

0

In [100]:
import pickle
# Dumping class_labels
with open('../Dataset/samples_df.pickle', 'wb') as file:
    pickle.dump(samples_df, file)
    print(f"Object successfully pickled and saved to '../Dataset/samples_df.pickle'! ✨")

Object successfully pickled and saved to '../Dataset/samples_df.pickle'! ✨


In [101]:
samples_df

,class_type,class_label,img_names
0,around,around01,"[around01_1.jpg, around01_2.jpg, around01_3.jp..."
1,around,around01,"[around01_2.jpg, around01_3.jpg, around01_4.jp..."
2,around,around02,"[around02_1.jpg, around02_2.jpg, around02_3.jp..."
3,around,around02,"[around02_2.jpg, around02_3.jpg, around02_4.jp..."
4,around,around02,"[around02_3.jpg, around02_4.jpg, around02_5.jp..."
...,...,...,...
1601,sleep,sleep30,"[sleep30_4.jpg, sleep30_5.jpg, sleep30_6.jpg, ..."
1602,sleep,sleep30,"[sleep30_5.jpg, sleep30_6.jpg, sleep30_7.jpg, ..."
1603,sleep,sleep30,"[sleep30_6.jpg, sleep30_7.jpg, sleep30_8.jpg, ..."
1604,sleep,sleep31,"[sleep31_1.jpg, sleep31_2.jpg, sleep31_3.jpg, ..."


In [7]:
x = pd.DataFrame(samples.keys())
x[0] = x[0].apply(lambda y: y[:-1])
x

,0
0,around01
1,around01
2,around02
3,around02
4,around02
...,...
1601,sleep30
1602,sleep30
1603,sleep30
1604,sleep31


In [8]:
x.value_counts()

hand02        10
hand016       10
read104       10
read11        10
hand10        10
              ..
read071        1
hand032        1
forward29      1
forward206     1
around127      1
Length: 193, dtype: int64

In [33]:
x.iloc[0]

0    around010
Name: 0, dtype: object

In [108]:
# import pickle
# # Dumping class_labels
# with open('../Dataset/samples_map.pickle', 'wb') as file:
#     pickle.dump(list(samples.values()), file)
#     print(f"Object successfully pickled and saved to '../Dataset/samples_map.pickle'! ✨")

Object successfully pickled and saved to '../Dataset/samples_map.pickle'! ✨


In [13]:
import pickle
# Dumping class_labels
with open('../Dataset/samples_dict.pickle', 'wb') as file:
    pickle.dump(samples, file)
    print(f"Object successfully pickled and saved to '../Dataset/samples_dict.pickle'! ✨")

Object successfully pickled and saved to '../Dataset/samples_dict.pickle'! ✨


In [11]:
samples

{'around010': ['around01_1.jpg',
  'around01_2.jpg',
  'around01_3.jpg',
  'around01_4.jpg',
  'around01_5.jpg',
  'around01_6.jpg',
  'around01_7.jpg',
  'around01_8.jpg'],
 'around011': ['around01_2.jpg',
  'around01_3.jpg',
  'around01_4.jpg',
  'around01_5.jpg',
  'around01_6.jpg',
  'around01_7.jpg',
  'around01_8.jpg',
  'around01_9.jpg'],
 'around020': ['around02_1.jpg',
  'around02_2.jpg',
  'around02_3.jpg',
  'around02_4.jpg',
  'around02_5.jpg',
  'around02_6.jpg',
  'around02_7.jpg',
  'around02_8.jpg'],
 'around021': ['around02_2.jpg',
  'around02_3.jpg',
  'around02_4.jpg',
  'around02_5.jpg',
  'around02_6.jpg',
  'around02_7.jpg',
  'around02_8.jpg',
  'around02_9.jpg'],
 'around022': ['around02_3.jpg',
  'around02_4.jpg',
  'around02_5.jpg',
  'around02_6.jpg',
  'around02_7.jpg',
  'around02_8.jpg',
  'around02_9.jpg',
  'around02_10.jpg'],
 'around023': ['around02_4.jpg',
  'around02_5.jpg',
  'around02_6.jpg',
  'around02_7.jpg',
  'around02_8.jpg',
  'around02_9.jp

In [10]:
class_labels

{'around01': ['around01_1.jpg',
  'around01_2.jpg',
  'around01_3.jpg',
  'around01_4.jpg',
  'around01_5.jpg',
  'around01_6.jpg',
  'around01_7.jpg',
  'around01_8.jpg',
  'around01_9.jpg',
  'around01_10.jpg'],
 'around02': ['around02_1.jpg',
  'around02_2.jpg',
  'around02_3.jpg',
  'around02_4.jpg',
  'around02_5.jpg',
  'around02_6.jpg',
  'around02_7.jpg',
  'around02_8.jpg',
  'around02_9.jpg',
  'around02_10.jpg',
  'around02_11.jpg',
  'around02_12.jpg',
  'around02_13.jpg',
  'around02_14.jpg',
  'around02_15.jpg',
  'around02_16.jpg',
  'around02_17.jpg',
  'around02_18.jpg',
  'around02_19.jpg',
  'around02_20.jpg',
  'around02_21.jpg',
  'around02_22.jpg',
  'around02_23.jpg',
  'around02_24.jpg',
  'around02_25.jpg',
  'around02_26.jpg',
  'around02_27.jpg',
  'around02_28.jpg',
  'around02_29.jpg',
  'around02_30.jpg',
  'around02_31.jpg',
  'around02_32.jpg',
  'around02_33.jpg',
  'around02_34.jpg',
  'around02_35.jpg',
  'around02_36.jpg',
  'around02_37.jpg',
  'aro